In [2]:
#id-2 - [1,0], id_7 - [0,1]
import tensorflow as tf 
# import tensorflow.examples.tutorials.mnist.input_data as input_data
import cv2

In [50]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import random

In [46]:
data_x, data_y = [], []

In [47]:
folder = 'pixiv/id_2'
for img_file in os.listdir(folder):
    img=np.array(Image.open(folder+'/'+img_file))
#     img=np.array(Image.open('/'.join([folder, img_file])))
#     img = np.reshape(img, 500*500*3)
    img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
#     print(len(img))
#     print(img)
    data_x.append(img/255.0)
    data_y.append([1,0])
print(len(data_x))
print(len(data_y))

200
200


In [48]:
folder = 'pixiv/id_7'
for img_file in os.listdir(folder):
    img=np.array(Image.open(folder+'/'+img_file))
#     img=np.array(Image.open('/'.join([folder, img_file])))
#     img = np.reshape(img, 500*500*3)
    img = np.reshape(img, len(img)*len(img[0])*len(img[0][0]))
#     print(len(img))
#     print(img)
    data_x.append(img/255.0)
    data_y.append([1,0])
print(len(data_x))
print(len(data_y))

400
400


In [49]:
print(data_x[:10])
print(data_y[:10])

[array([ 0.98039216,  0.98431373,  1.        , ...,  0.9254902 ,
        0.99607843,  1.        ]), array([ 0.56862745,  0.78431373,  1.        , ...,  0.98039216,
        0.98431373,  0.92941176]), array([ 0.48627451,  0.80392157,  1.        , ...,  0.94117647,
        0.99215686,  0.96470588]), array([ 0.99215686,  0.99215686,  0.99215686, ...,  0.97647059,
        0.97254902,  0.95686275]), array([ 0.82745098,  0.90980392,  0.99215686, ...,  0.83921569,
        0.81568627,  0.76862745]), array([ 0.34901961,  0.4       ,  0.29019608, ...,  0.94509804,
        0.96862745,  0.96078431]), array([ 0.76862745,  0.83921569,  0.98823529, ...,  0.94901961,
        0.98823529,  0.99607843]), array([ 0.90980392,  0.94509804,  0.98039216, ...,  0.83529412,
        0.83921569,  0.80784314]), array([ 0.78431373,  0.76078431,  0.69803922, ...,  0.42745098,
        0.41960784,  0.37254902]), array([ 0.80392157,  0.81960784,  0.82352941, ...,  0.99607843,
        1.        ,  1.        ])]
[[1, 0], 

In [56]:
order = [i for i in range(len(data_x))]
random.shuffle(order)

print(len(order))
n=len(data_x)/10*9
print(n)

train_x = [data_x[i] for i in order[:n]]
train_y = [data_y[i] for i in order[:n]]

test_x = [data_x[i] for i in order[n:]]
test_y = [data_y[i] for i in order[n:]]

print(len(train_x))
print(len(test_y))

400
360
360
40


In [57]:
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)     #下载并加载mnist数据
x = tf.placeholder(tf.float32, [None, 500*500*3])                        #输入的数据占位符
y_actual = tf.placeholder(tf.float32, shape=[None, 2])            #输入的标签占位符

#定义一个函数，用于初始化所有的权值 W
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

#定义一个函数，用于初始化所有的偏置项 b
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)
  
#定义一个函数，用于构建卷积层
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

#定义一个函数，用于构建池化层
def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')

"""
# 第一层
# 卷积核(filter)的尺寸是5*5, 通道数为3，输出通道为32，即feature map 数目为32
# 又因为strides=[1,1,1,1] 所以单个通道的输出尺寸应该跟输入图像一样。即总的卷积输出应该为?*500*500*32
# 也就是单个通道输出为500*500，共有32个通道,共有?个批次
# 在池化阶段，ksize=[1,2,2,1] 那么卷积结果经过池化以后的结果，其尺寸应该是？*250*250*32
"""

#构建网络
x_image = tf.reshape(x, [-1,500,500,3])         #转换输入数据shape,以便于用于网络中
W_conv1 = weight_variable([5, 5, 3, 32])      
b_conv1 = bias_variable([32])       
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)     #第一个卷积层
h_pool1 = max_pool(h_conv1)                                  #第一个池化层

"""
# 第二层
# 卷积核5*5，输入通道为32，输出通道为64。
# 卷积前图像的尺寸为 ?*250*250*32， 卷积后为?*250*250*64
# 池化后，输出的图像尺寸为?*125*125*64
"""

W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)      #第二个卷积层
h_pool2 = max_pool(h_conv2)                                   #第二个池化层

# 第三层 是个全连接层,输入维数125*125*64, 输出维数为1024
W_fc1 = weight_variable([125 * 125 * 64, 1024])
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 125*125*64])              #reshape成向量
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)    #第一个全连接层

keep_prob = tf.placeholder("float") 
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)                  #dropout层

W_fc2 = weight_variable([1024, 2])
b_fc2 = bias_variable([2])
y_predict=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)   #softmax层

cross_entropy = -tf.reduce_sum(y_actual*tf.log(y_predict))     #交叉熵
train_step = tf.train.GradientDescentOptimizer(1e-3).minimize(cross_entropy)    #梯度下降法
correct_prediction = tf.equal(tf.argmax(y_predict,1), tf.argmax(y_actual,1))    
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))                 #精确度计算
sess=tf.InteractiveSession()                          
sess.run(tf.global_variables_initializer())

batch_start = 0
batch_size = 50
for i in range(20000):
    #batch = mnist.train.next_batch(50)
    if(batch_start+batch_size>=len(train_x)):
        order = [i for i in range(len(train_x))]
        random.shuffle(order)
        train_x = [train_x[i] for i in order]
        train_y = [train_y[i] for i in order]
        batch_start = 0
        
    feed_dict = {X: train_x[batch_start:batch_start+batch_size], Y: train_y[batch_start:batch_start+batch_size]}
    _, l, predictions= session.run([optimizer, loss, logits], feed_dict = feed_dict)
        
    if i%100 == 0:                  #训练100次，验证一次
        train_acc = accuracy.eval(feed_dict={x:batch[0], y_actual: batch[1], keep_prob: 1.0})
        print('step',i,'training accuracy',train_acc)
        train_step.run(feed_dict={x: batch[0], y_actual: batch[1], keep_prob: 0.5})

test_acc=accuracy.eval(feed_dict={x: mnist.test.images, y_actual: mnist.test.labels, keep_prob: 1.0})
print("test accuracy",test_acc)

KeyboardInterrupt: 